In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfs = []
files = glob.glob("/scratch/users/schuetzn/data/mhc_dataset/*/metadata.parquet")
for file in files:
    hq = file.split("/")[-2]
    df = pd.read_parquet(file)
    df["healthCode"] = hq
    dfs.append(df)


In [3]:
df = pd.concat(dfs)

In [4]:
daily_stats_per_modality = []
for idx, row in df.groupby(["healthCode", "date"]):
    # Common data for all entries
    healthCode = row["healthCode"].iloc[0]
    date = row["date"].iloc[0]
    
    # Map specific variables to their corresponding indices in the dataframe
    variable_mappings = {
        "WorkoutsAny": [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],  # All workout types
        "WorkoutsRunning": [16],  # HKWorkoutActivityTypeRunning
        "WorkoutsCycling": [15],  # HKWorkoutActivityTypeCycling
        "HeartRate": [5],  # HKQuantityTypeIdentifierHeartRate
        "StepsTaken": [0],  # HKQuantityTypeIdentifierStepCount
        "DistanceWalkingRunning": [2],  # HKQuantityTypeIdentifierDistanceWalkingRunning
        "DistanceCycling": [3],  # HKQuantityTypeIdentifierDistanceCycling
        "ActiveCaloriesBurned": [1],  # HKQuantityTypeIdentifierActiveEnergyBurned
        "SleepAny": [12, 13]  # Both sleep types - Asleep and InBed
    }
    
    # Create entry for this healthCode/date combination
    entry = {
        "healthCode": healthCode,
        "date": date
    }
    
    # Calculate data_coverage_sum for each requested variable
    for variable_name, indices in variable_mappings.items():
        # For variables that map to multiple indices, sum the data_coverage across those indices
        if len(indices) > 1:
            coverage_sum = row.iloc[indices].data_coverage.sum()
        else:
            coverage_sum = row.iloc[indices[0]].data_coverage.sum()
        
        entry[f"{variable_name}_coverage_sum"] = coverage_sum
    
    daily_stats_per_modality.append(entry)


In [ ]:
len(daily_stats_per_modality)

In [2]:
import pickle
#with open("../per_modality_daily.pkl", "wb") as f:
#    pickle.dump(daily_stats_per_modality, f)
with open("../per_modality_daily.pkl", "rb") as f:
    daily_stats_per_modality = pickle.load(f)

In [4]:
coverage_by_modality_df = pd.DataFrame(daily_stats_per_modality)

In [ ]:
# Create a dictionary to store the results
modality_stats = {}

# Get the list of modalities (excluding healthCode and date columns)
modalities = [col for col in coverage_by_modality_df.columns if col not in ['healthCode', 'date']]

# For each modality, calculate days > 0, unique participants, and estimated data points
for modality in modalities:
    # Filter to only rows where coverage > 0 for this modality
    covered_data = coverage_by_modality_df[coverage_by_modality_df[modality] > 0]
    
    # Count unique days and participants
    days_count = len(covered_data)
    participants_count = covered_data['healthCode'].nunique()
    
    # Calculate estimated total data points (coverage of 100 = 1440 data points)
    total_coverage = coverage_by_modality_df[modality].sum()
    estimated_data_points = total_coverage * (1440/100)
    
    # Store in results dictionary
    modality_stats[modality] = {
        'days_with_coverage': days_count,
        'unique_participants': participants_count,
        'estimated_data_points': int(estimated_data_points)  # Convert to integer for cleaner display
    }

# Convert to a DataFrame for better display
result_df = pd.DataFrame.from_dict(modality_stats, orient='index')

# Rename the modalities for clarity
modality_names = {
    'WorkoutsAny_coverage_sum': 'Workouts (Any Type)',
    'WorkoutsRunning_coverage_sum': 'Workouts - Running',
    'WorkoutsCycling_coverage_sum': 'Workouts - Cycling',
    'HeartRate_coverage_sum': 'Heart Rate',
    'StepsTaken_coverage_sum': 'Steps Taken',
    'DistanceWalkingRunning_coverage_sum': 'Distance Walking/Running',
    'DistanceCycling_coverage_sum': 'Distance Cycling',
    'ActiveCaloriesBurned_coverage_sum': 'Active Calories Burned',
    'SleepAny_coverage_sum': 'Sleep (Any Type)'
}

# Create a new dataframe with clear modality names
clear_result_df = result_df.copy()
clear_result_df.index = [modality_names[idx] for idx in clear_result_df.index]

# Format the estimated data points with commas for readability
clear_result_df['estimated_data_points'] = clear_result_df['estimated_data_points'].apply(lambda x: f"{x:,}")

# Display the results
print("Coverage statistics for each modality:")
print(clear_result_df)

# Optionally, sort by number of days with coverage to see most common modalities
print("\nModalities sorted by coverage (most to least):")
print(clear_result_df.sort_values('days_with_coverage', ascending=False))

In [4]:
stats = df.groupby("healthCode")["date"].apply(lambda x: len(np.unique(x)))

In [ ]:
counts_series = coverage_df.groupby("healthCode").data_coverage_sum.count()
thresholds = [1, 3, 7, 14, 30, 365, 5 * 365]
values = [(counts_series > t).sum() for t in thresholds]
counts = [(counts_series > t).sum() for t in thresholds]


sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=counts, palette="viridis")
for i, count in enumerate(counts):
    ax.text(i, count + max(counts) * 0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Number of rows with count > threshold")
plt.title("Rows with counts above various thresholds")
plt.show()

In [6]:
daily_stats = []
for idx, row in df.groupby(["healthCode", "date"]):
    data_coverage_sum = row.data_coverage.sum()
    data_coverage_mean = row.data_coverage.mean()
    n_above_zero = row[row.data_coverage > 0]
    daily_stats.append({
        "data_coverage_sum": data_coverage_sum,
        "data_coverage_mean": data_coverage_mean,
        "n_above_zero": len(n_above_zero),
        "healthCode": row["healthCode"].iloc[0]
    })

In [ ]:
data_coverage_mean

In [ ]:
len(daily_stats)

In [ ]:
daily_stats

In [1]:
import pickle
#with open("../summary_daily.pkl", "wb") as f:
#    pickle.dump(daily_stats, f)
with open("../summary_daily.pkl", "rb") as f:
    daily_stats = pickle.load(f)

In [ ]:
coverage_df.groupby("healthCode").data_coverage_sum.count()

In [3]:
info_df = pd.read_parquet('../myheart-counts-client/mhc_full_participant_info.parquet')

In [4]:
healthCodes_open_sharing = set(info_df[info_df.sharingScope == "all_qualified_researchers"].healthCode)

In [ ]:
healthCodes_open_sharing

In [ ]:
coverage_df = pd.DataFrame(daily_stats)
len(coverage_df)

In [7]:
coverage_df = coverage_df[coverage_df.healthCode.isin(healthCodes_open_sharing)]

## Days with ANY data

In [ ]:
len(coverage_df[coverage_df.data_coverage_sum > 0])

In [ ]:
coverage_df[coverage_df.data_coverage_sum > 0].healthCode.unique().shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

thresholds = [2, 3, 5, 8, 10, 15, 24]
counts = [len(coverage_df[coverage_df.n_above_zero >= t]) for t in thresholds]

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=counts, palette="viridis")
for i, count in enumerate(counts):
    ax.text(i, count + max(counts)*0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Number of rows with at least k number of modalities")
#plt.title("Rows with n_above_zero above various thresholds")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

thresholds = [2, 3, 5, 8, 10, 15, 24]
unique_counts = [coverage_df[coverage_df.n_above_zero > t]['healthCode'].nunique() for t in thresholds]

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=unique_counts, palette="viridis")
for i, count in enumerate(unique_counts):
    ax.text(i, count + max(unique_counts)*0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Unique healthCode count")
plt.title("Unique healthCodes for rows with n_above_zero > threshold")
plt.show()